In [1]:
import torch
import torchvision
import torch.nn
import numpy as np
import random
import os
import time
from torch.utils.data import DataLoader

In [2]:
# 固定随机数种子，保证模型的可复现性
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    
set_seed(666)

In [3]:
#超参数
epochs = 5
batch_size = 64

In [4]:
# 预处理
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5,),(0.5,))]     )

# 加载数据集
train_data = torchvision.datasets.MNIST(root = './data', train = True, download = True, transform = transform)    
test_data = torchvision.datasets.MNIST(root = './data', train = False, download = True, transform = transform)      

train_dl = DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_dl = DataLoader(test_data, batch_size = batch_size, shuffle = True)

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.fc1 = torch.nn.Linear(784, 500)
        self.relu2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(500, 200)
        self.relu4 = torch.nn.ReLU()
        self.fc5 = torch.nn.Linear(200, 10)
 
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu4(x)
        x = self.fc5(x)
        return x

model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=500, out_features=200, bias=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=200, out_features=10, bias=True)
)


In [6]:
# 训练
time1 = time.clock()
for epoch in range(epochs):
    for i, (images, lables) in enumerate(train_dl):
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds, lables)
        loss.backward()
        optimizer.step()
 
        if i % 200 ==0:
            print('epoch={}, batch={}, loss={:g}'.format(epoch, i, loss.item()))
time2 = time.clock()
print(time2 - time1)

F:\Ksoftware\anacondanew\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


epoch=0, batch=0, loss=2.29711
epoch=0, batch=200, loss=0.386776
epoch=0, batch=400, loss=0.192278
epoch=0, batch=600, loss=0.259303
epoch=0, batch=800, loss=0.273769
epoch=1, batch=0, loss=0.0951458
epoch=1, batch=200, loss=0.15161
epoch=1, batch=400, loss=0.171003
epoch=1, batch=600, loss=0.208048
epoch=1, batch=800, loss=0.0959579
epoch=2, batch=0, loss=0.104961
epoch=2, batch=200, loss=0.123491
epoch=2, batch=400, loss=0.0975131
epoch=2, batch=600, loss=0.051745
epoch=2, batch=800, loss=0.022118
epoch=3, batch=0, loss=0.0911821
epoch=3, batch=200, loss=0.0202966
epoch=3, batch=400, loss=0.0158767
epoch=3, batch=600, loss=0.185679
epoch=3, batch=800, loss=0.0609254
epoch=4, batch=0, loss=0.0376248
epoch=4, batch=200, loss=0.126354
epoch=4, batch=400, loss=0.0199629
epoch=4, batch=600, loss=0.0492025
epoch=4, batch=800, loss=0.0661325
167.4150154


F:\Ksoftware\anacondanew\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


In [7]:
#测试准确性
total = 0
correct = 0

for test_image, test_label in test_dl:
    test_pred = model(test_image)
    pred = torch.argmax(test_pred, 1)
    total += test_label.size(0)
    correct += (pred == test_label).sum().item()

#打印准确率
print(correct / total)

0.9751
